# utils

In [1]:
import vectorbt as vbt

from vectorbt.utils import checks, config, decorators

In [2]:
import numpy as np
import pandas as pd
from numba import njit

In [3]:
v1 = 0
a1 = np.array([1])
a2 = np.array([1, 2, 3])
a3 = np.array([[1, 2, 3]])
a4 = np.array([[1], [2], [3]])
a5 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
sr1 = pd.Series([1], index=pd.Index(['x1'], name='i1'))
print(sr1)
sr2 = pd.Series([1, 2, 3], index=pd.Index(['x2', 'y2', 'z2'], name='i2'))
print(sr2)
df1 = pd.DataFrame(
    [[1]], 
    index=pd.Index(['x3'], name='i3'), 
    columns=pd.Index(['a3'], name='c3'))
print(df1)
df2 = pd.DataFrame(
    [[1], [2], [3]], 
    index=pd.Index(['x4', 'y4', 'z4'], name='i4'), 
    columns=pd.Index(['a4'], name='c4'))
print(df2)
df3 = pd.DataFrame(
    [[1, 2, 3]], 
    index=pd.Index(['x5'], name='i5'), 
    columns=pd.Index(['a5', 'b5', 'c5'], name='c5'))
print(df3)
df4 = pd.DataFrame(
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]], 
    index=pd.Index(['x6', 'y6', 'z6'], name='i6'), 
    columns=pd.Index(['a6', 'b6', 'c6'], name='c6'))
print(df4)

multi_i = pd.MultiIndex.from_arrays([['x7', 'y7', 'z7'], ['x8', 'y8', 'z8']], names=['i7', 'i8']) 
multi_c = pd.MultiIndex.from_arrays([['a7', 'b7', 'c7'], ['a8', 'b8', 'c8']], names=['c7', 'c8'])
df5 = pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], index=multi_i, columns=multi_c)
print(df5)

i1
x1    1
dtype: int64
i2
x2    1
y2    2
z2    3
dtype: int64
c3  a3
i3    
x3   1
c4  a4
i4    
x4   1
y4   2
z4   3
c5  a5  b5  c5
i5            
x5   1   2   3
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  2  3
y7 y8  4  5  6
z7 z8  7  8  9


## config

In [4]:
conf = config.Config({'a': 0, 'b': {'c': 1}}, frozen=False)
conf['b']['d'] = 2

conf = config.Config({'a': 0, 'b': {'c': 1}}, frozen=True)
conf['a'] = 2

try:
    conf['d'] = 2
except Exception as e:
    print(e)

try:
    conf.update(d=2)
except Exception as e:
    print(e)
    
conf.update(d=2, force_update=True)

"Key 'd' is not valid"
"Key 'd' is not valid"


In [5]:
conf = config.Config({'a': 0, 'b': {'c': 1}}, read_only=True)

try:
    conf['a'] = 2
except Exception as e:
    print(e)
    
try:
    del conf['a']
except Exception as e:
    print(e)
    
try:
    conf.pop('a')
except Exception as e:
    print(e)
    
try:
    conf.popitem()
except Exception as e:
    print(e)
    
try:
    conf.clear()
except Exception as e:
    print(e)
    
try:
    conf.update(a=2)
except Exception as e:
    print(e)
    
print(conf.merge_with(dict(b=dict(d=2))))

Config is read-only
Config is read-only
Config is read-only
Config is read-only
Config is read-only
Config is read-only
{'b': {'c': 1, 'd': 2}, 'a': 0}


In [6]:
print(config.merge_dicts({'a': 1}, {'b': 2}))
print(config.merge_dicts({'a': 1}, {'a': 2}))
print(config.merge_dicts({'a': {'b': 2}}, {'a': {'c': 3}}))
print(config.merge_dicts({'a': {'b': 2}}, {'a': {'b': 3}}))

{'a': 1, 'b': 2}
{'a': 2}
{'a': {'b': 2, 'c': 3}}
{'a': {'b': 3}}


In [7]:
class H(config.Configured):
    def __init__(self, a, b=2, **kwargs):
        super().__init__(a=a, b=b, **kwargs)
        
print(H(1).config)
print(H(1).copy(b=3).config)
print(H(1).copy(c=4).config)

{'a': 1, 'b': 2}
{'a': 1, 'b': 3}
{'a': 1, 'b': 2, 'c': 4}


## decorators

In [8]:
class G():
    @decorators.class_or_instancemethod
    def g(self_or_cls):
        if isinstance(self_or_cls, type):
            print("class")
        else:
            print("instance")
            
G.g()
G().g()

class
instance


In [9]:
class G():
    @decorators.cached_property(hello="world", hello2="world2")
    def cache_me(self): return np.random.uniform(size=(10000, 10000))
    
G.cache_me.kwargs

{'hello': 'world', 'hello2': 'world2'}

In [10]:
g = G()

In [22]:
%time _ = g.cache_me
%time _ = g.cache_me

CPU times: user 1.25 s, sys: 310 ms, total: 1.56 s
Wall time: 1.58 s
CPU times: user 61 µs, sys: 5 µs, total: 66 µs
Wall time: 82.7 µs


In [11]:
dir(g)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cache_me']

In [13]:
G.cache_me.clear_cache(g)
%time _ = g.cache_me
%time _ = g.cache_me

CPU times: user 1.28 s, sys: 264 ms, total: 1.55 s
Wall time: 1.6 s
CPU times: user 18 µs, sys: 0 ns, total: 18 µs
Wall time: 20.3 µs


In [12]:
G.cache_me.clear_cache(g)
G.cache_me.disabled = True
%time _ = g.cache_me
%time _ = g.cache_me
G.cache_me.disabled = False

CPU times: user 1.23 s, sys: 231 ms, total: 1.46 s
Wall time: 1.49 s
CPU times: user 1.25 s, sys: 289 ms, total: 1.54 s
Wall time: 1.62 s


In [11]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append((g, 'cache_me'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.32 s, sys: 272 ms, total: 1.59 s
Wall time: 1.82 s
CPU times: user 1.3 s, sys: 350 ms, total: 1.65 s
Wall time: 1.71 s
{'enabled': True, 'whitelist': [], 'blacklist': []}


In [12]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('cache_me')
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.26 s, sys: 333 ms, total: 1.6 s
Wall time: 1.65 s
CPU times: user 1.27 s, sys: 317 ms, total: 1.59 s
Wall time: 1.68 s
{'enabled': True, 'whitelist': [], 'blacklist': []}


In [13]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(g)
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.3 s, sys: 356 ms, total: 1.65 s
Wall time: 1.77 s
CPU times: user 1.28 s, sys: 335 ms, total: 1.61 s
Wall time: 1.71 s
{'enabled': True, 'whitelist': [], 'blacklist': []}


In [14]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(G)
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.28 s, sys: 328 ms, total: 1.61 s
Wall time: 1.67 s
CPU times: user 1.26 s, sys: 322 ms, total: 1.58 s
Wall time: 1.65 s
{'enabled': True, 'whitelist': [], 'blacklist': []}


In [15]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('G')
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.28 s, sys: 341 ms, total: 1.62 s
Wall time: 1.71 s
CPU times: user 1.22 s, sys: 309 ms, total: 1.53 s
Wall time: 1.55 s
{'enabled': True, 'whitelist': [], 'blacklist': []}


In [19]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('g')
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.22 s, sys: 288 ms, total: 1.51 s
Wall time: 1.53 s
CPU times: user 20 µs, sys: 0 ns, total: 20 µs
Wall time: 21.9 µs
{'enabled': True, 'whitelist': [], 'blacklist': []}


In [16]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world'})
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.29 s, sys: 332 ms, total: 1.62 s
Wall time: 1.73 s
CPU times: user 1.33 s, sys: 387 ms, total: 1.71 s
Wall time: 1.97 s
{'enabled': True, 'whitelist': [], 'blacklist': []}


In [17]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world', 'hello2': 'world2'})
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.31 s, sys: 370 ms, total: 1.68 s
Wall time: 1.87 s
CPU times: user 1.34 s, sys: 368 ms, total: 1.7 s
Wall time: 1.82 s
{'enabled': True, 'whitelist': [], 'blacklist': []}


In [18]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world', 'hello2': 'world2', 'hello3': 'world3'})
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.25 s, sys: 312 ms, total: 1.56 s
Wall time: 1.59 s
CPU times: user 20 µs, sys: 0 ns, total: 20 µs
Wall time: 22.9 µs
{'enabled': True, 'whitelist': [], 'blacklist': []}


In [15]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.28 s, sys: 367 ms, total: 1.64 s
Wall time: 1.91 s
CPU times: user 1.23 s, sys: 339 ms, total: 1.57 s
Wall time: 1.73 s
{'enabled': True, 'whiteset': set(), 'blackset': set()}


In [23]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append((g, 'cache_me'))
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.22 s, sys: 285 ms, total: 1.51 s
Wall time: 1.53 s
CPU times: user 17 µs, sys: 1 µs, total: 18 µs
Wall time: 20 µs


In [24]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append('cache_me')
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 288 ms, total: 1.52 s
Wall time: 1.54 s
CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 18.8 µs


In [25]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(g)
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 306 ms, total: 1.54 s
Wall time: 1.56 s
CPU times: user 17 µs, sys: 1 µs, total: 18 µs
Wall time: 18.8 µs


In [26]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(G)
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.23 s, sys: 300 ms, total: 1.53 s
Wall time: 1.54 s
CPU times: user 23 µs, sys: 0 ns, total: 23 µs
Wall time: 24.8 µs


In [27]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append('G')
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.28 s, sys: 325 ms, total: 1.6 s
Wall time: 1.7 s
CPU times: user 31 µs, sys: 0 ns, total: 31 µs
Wall time: 36.2 µs


In [28]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append({'hello': 'world'})
%time _ = g.cache_me
%time _ = g.cache_me
vbt.settings.caching.reset()

CPU times: user 1.24 s, sys: 341 ms, total: 1.58 s
Wall time: 1.92 s
CPU times: user 22 µs, sys: 0 ns, total: 22 µs
Wall time: 24.1 µs


In [5]:
class G():
    @decorators.cached_method(hello="world", hello2="world2")
    def cache_me(self, a): return np.random.uniform(size=(10000, 10000)) * a

G.cache_me.kwargs

{'hello': 'world', 'hello2': 'world2'}

In [6]:
g = G()

In [34]:
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)

CPU times: user 1.37 s, sys: 301 ms, total: 1.67 s
Wall time: 1.73 s
CPU times: user 22 µs, sys: 0 ns, total: 22 µs
Wall time: 24.8 µs


In [6]:
dir(g)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cache_me']

In [21]:
G.cache_me.clear_cache(g)
%time _ = g.cache_me(2)

CPU times: user 1.82 s, sys: 431 ms, total: 2.25 s
Wall time: 3.03 s


In [33]:
G.cache_me.clear_cache(g)
G.cache_me.disabled = True
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
G.cache_me.disabled = False

CPU times: user 1.43 s, sys: 340 ms, total: 1.77 s
Wall time: 2.01 s
CPU times: user 1.41 s, sys: 349 ms, total: 1.76 s
Wall time: 1.91 s


In [6]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(g.cache_me)
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.32 s, sys: 229 ms, total: 1.55 s
Wall time: 1.57 s
CPU times: user 1.38 s, sys: 287 ms, total: 1.67 s
Wall time: 1.68 s


In [7]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append((g, 'cache_me'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.44 s, sys: 274 ms, total: 1.72 s
Wall time: 1.85 s
CPU times: user 1.47 s, sys: 331 ms, total: 1.81 s
Wall time: 2.05 s


In [8]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('cache_me')
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.42 s, sys: 343 ms, total: 1.76 s
Wall time: 1.88 s
CPU times: user 1.38 s, sys: 322 ms, total: 1.7 s
Wall time: 1.81 s


In [9]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(g)
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.44 s, sys: 341 ms, total: 1.78 s
Wall time: 1.96 s
CPU times: user 1.31 s, sys: 288 ms, total: 1.6 s
Wall time: 1.6 s


In [10]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append(G)
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.43 s, sys: 342 ms, total: 1.77 s
Wall time: 2 s
CPU times: user 1.32 s, sys: 296 ms, total: 1.61 s
Wall time: 1.62 s


In [11]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('G')
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.46 s, sys: 358 ms, total: 1.82 s
Wall time: 1.99 s
CPU times: user 1.31 s, sys: 296 ms, total: 1.61 s
Wall time: 1.61 s


In [12]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append('g')
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.38 s, sys: 326 ms, total: 1.7 s
Wall time: 1.76 s
CPU times: user 27 µs, sys: 1e+03 ns, total: 28 µs
Wall time: 30.3 µs


In [13]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world'})
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.36 s, sys: 305 ms, total: 1.67 s
Wall time: 1.73 s
CPU times: user 1.39 s, sys: 333 ms, total: 1.72 s
Wall time: 1.77 s


In [14]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world', 'hello2': 'world2'})
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.47 s, sys: 353 ms, total: 1.82 s
Wall time: 2.01 s
CPU times: user 1.44 s, sys: 356 ms, total: 1.8 s
Wall time: 1.96 s


In [15]:
G.cache_me.clear_cache(g)
vbt.settings.caching['blacklist'].append({'hello': 'world', 'hello2': 'world2', 'hello3': 'world3'})
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.42 s, sys: 346 ms, total: 1.76 s
Wall time: 1.88 s
CPU times: user 29 µs, sys: 1 µs, total: 30 µs
Wall time: 31 µs


In [16]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.34 s, sys: 307 ms, total: 1.65 s
Wall time: 1.67 s
CPU times: user 1.36 s, sys: 334 ms, total: 1.7 s
Wall time: 1.76 s


In [17]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append((g, 'cache_me'))
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.43 s, sys: 341 ms, total: 1.77 s
Wall time: 1.89 s
CPU times: user 23 µs, sys: 1 µs, total: 24 µs
Wall time: 26.2 µs


In [18]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append('cache_me')
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.45 s, sys: 361 ms, total: 1.81 s
Wall time: 1.97 s
CPU times: user 23 µs, sys: 1 µs, total: 24 µs
Wall time: 26.2 µs


In [19]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(g)
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.45 s, sys: 368 ms, total: 1.82 s
Wall time: 2 s
CPU times: user 24 µs, sys: 0 ns, total: 24 µs
Wall time: 26.9 µs


In [20]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append(G)
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.51 s, sys: 362 ms, total: 1.88 s
Wall time: 2.17 s
CPU times: user 24 µs, sys: 1 µs, total: 25 µs
Wall time: 26.9 µs


In [21]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append('G')
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.44 s, sys: 367 ms, total: 1.81 s
Wall time: 2.1 s
CPU times: user 26 µs, sys: 1e+03 ns, total: 27 µs
Wall time: 28.8 µs


In [22]:
G.cache_me.clear_cache(g)
vbt.settings.caching['enabled'] = False
vbt.settings.caching['whitelist'].append({'hello': 'world'})
%time _ = g.cache_me(2)
%time _ = g.cache_me(2)
vbt.settings.caching.reset()

CPU times: user 1.34 s, sys: 298 ms, total: 1.64 s
Wall time: 1.67 s
CPU times: user 27 µs, sys: 1e+03 ns, total: 28 µs
Wall time: 30 µs


In [24]:
# Non-hashable arguments won't cache
%time _ = g.cache_me(np.asarray(2))
%time _ = g.cache_me(np.asarray(2))

CPU times: user 1.36 s, sys: 320 ms, total: 1.68 s
Wall time: 1.7 s
CPU times: user 1.39 s, sys: 312 ms, total: 1.7 s
Wall time: 1.81 s


In [25]:
class A:
    @decorators.custom_property(some_key=0)
    def a(self): pass

class B:
    @decorators.cached_property(some_key=0, child_cls=A)
    def a(self): pass
    
    @decorators.custom_method(some_key=1)
    def b(self): pass
    
class C:
    @decorators.cached_method(some_key=0, child_cls=B)
    def b(self): pass
    
    @decorators.custom_property(some_key=1)
    def c(self): pass

In [26]:
str(decorators.traverse_attr_kwargs(C))

"{'b': {'some_key': 0, 'child_cls': <class '__main__.B'>, 'child_attrs': {'a': {'some_key': 0, 'child_cls': <class '__main__.A'>, 'child_attrs': {'a': {'some_key': 0}}}, 'b': {'some_key': 1}}}, 'c': {'some_key': 1}}"

In [27]:
decorators.traverse_attr_kwargs(C, key='some_key')

{'b': {'some_key': 0,
  'child_cls': __main__.B,
  'child_attrs': {'a': {'some_key': 0,
    'child_cls': __main__.A,
    'child_attrs': {'a': {'some_key': 0}}},
   'b': {'some_key': 1}}},
 'c': {'some_key': 1}}

In [28]:
decorators.traverse_attr_kwargs(C, key='some_key', value=1)

{'b': {'some_key': 0,
  'child_cls': __main__.B,
  'child_attrs': {'a': {'some_key': 0,
    'child_cls': __main__.A,
    'child_attrs': {}},
   'b': {'some_key': 1}}},
 'c': {'some_key': 1}}

In [29]:
decorators.traverse_attr_kwargs(C, key='some_key', value=(0, 1))

{'b': {'some_key': 0,
  'child_cls': __main__.B,
  'child_attrs': {'a': {'some_key': 0,
    'child_cls': __main__.A,
    'child_attrs': {'a': {'some_key': 0}}},
   'b': {'some_key': 1}}},
 'c': {'some_key': 1}}

## checks

In [30]:
print(checks.is_series(v1))
print(checks.is_series(a1))
print(checks.is_series(sr1))
print(checks.is_series(df1))

False
False
True
False


In [31]:
print(checks.is_frame(v1))
print(checks.is_frame(a1))
print(checks.is_frame(sr1))
print(checks.is_frame(df1))

False
False
False
True


In [32]:
print(checks.is_pandas(v1))
print(checks.is_pandas(a1))
print(checks.is_pandas(sr1))
print(checks.is_pandas(df1))

False
False
True
True


In [33]:
print(checks.is_array(v1))
print(checks.is_array(a1))
print(checks.is_array(sr1))
print(checks.is_array(df1))

False
True
True
True


In [34]:
print(checks.is_numba_func(lambda x: x))
print(checks.is_numba_func(njit(lambda x: x)))

False
True


In [35]:
print(checks.is_hashable(2))
print(checks.is_hashable(np.asarray(2)))

True
False


In [36]:
checks.assert_in(0, (0, 1))

In [37]:
checks.assert_numba_func(njit(lambda x: x))

In [38]:
checks.assert_not_none(v1)

In [39]:
checks.assert_type(v1, int)
checks.assert_type(a1, np.ndarray)
checks.assert_type(sr1, (np.ndarray, pd.Series))

In [40]:
checks.assert_type_equal(v1, v1)
checks.assert_type_equal(a1, a2)
checks.assert_type_equal(sr1, sr1)
checks.assert_type_equal(df1, df2)

In [41]:
checks.assert_dtype(a1, np.int)

In [42]:
checks.assert_dtype_equal(v1, a1)
checks.assert_dtype_equal(a1, df1)
checks.assert_dtype_equal(df1, df2)
checks.assert_dtype_equal(df2, df3)

In [43]:
checks.assert_ndim(v1, 0)
checks.assert_ndim(a1, 1)
checks.assert_ndim(df1, 2)

In [44]:
checks.assert_len_equal([[1]], [[2]])

In [45]:
checks.assert_shape_equal(a1, sr1)
checks.assert_shape_equal(df2, df4, axis=0)
checks.assert_shape_equal(df3, df4, axis=1)
checks.assert_shape_equal(df2, df3, axis=(0, 1))

In [46]:
checks.assert_index_equal(df3.index, df3.index)

In [47]:
checks.assert_meta_equal(df3, df3)

In [48]:
checks.assert_array_equal(df3, df3)

In [49]:
checks.assert_level_not_exists(df3.columns, 'a')